<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Linear Opamp 02 - Follow me

This document explains the **follower circuit**.  
That humble circuit yields a lot of information about opamp **stability** concepts.

Version 1.1 (1/6/2018)  
License information is at the end of the document

---

**Bill Of Materials (BOM):**

* 1x Dual Opamp MCP6002

---

## The Follower Circuit

In the previous document we have seen the opamp operate in **open loop** configuration. Now we will see the follower circuit. In this circuit, an external input $V_i$, generated by DAC 1, is connected to the positive input (+) and the output of the opamp $V_O$, read by ADC 1, is feedback to the negative input (-). We also add an ADC 2 connection to the input node $V_i$ so that we can measure the exact voltage applied on the circuit.

Note that the $V_{DD}$ supply is not shown using a voltage source to simplifly the schematic.

![fig 01](images\L_OA_02\fig01b.png)

---

**BUILD TASK**  
Mount the above circuit  
Don't forget to include the DAC and ADC connections  

---

We will try to deduce the behavior of the circuit using the zero order model.

![fig 02](images\L_OA_02\fig02.png)

We know that the zero order model is not good enough to model a real opamp because of saturation. However, we can rely on the model result as long as it gives a result between the saturation voltages. So, if using the zero order model we obtain a solution that satisfies:

$\qquad V_{O\:min} < V_O < V_{O\:min}$

Then we know that the more complex saturation model would have given the same result.
That’s a typical method to work in engineering:

1. We take the easier model we can find. This model relates to a list of requirements to meet so that the results are valid.

2. We use the model to solve a problem.

3. We check that the solution meets all the requirements of the model.


Sometimes you are tempted to perform steps 1 and 2 and neglect to perform step 3. Using a model outside of its requirements can give us completely **wrong** results. So never neglect the third step.

In the case of the zero order model the model equations and the requirements are:

|&nbsp;Model equations&nbsp;|&nbsp;Model Requirements&nbsp;|
|:---:|:---:|
|$V_d = V_{(+)}-V_{(-)}$|$V_O > V_{O\:min}$|
|$V_O = A \cdot V_d$|$V_O < V_{O\:min}$|

We have a model and we have a circuit so we can solve the Vo(Vi).  
From the circuit:

$\qquad V_{(+)} = V_i \qquad V_{(-)} = V_O$

Adding the model equations we get:

$\qquad V_O = A(V_i-V_O)$

We need to isolate **$V_O$** on the above equation.

---

**THEORETICAL TASK**  
Calculate Vo(Vi) from the above equation. It can only depend on $A$ and $Vi$.    
Obtain the result for an ideal opamp that satisfies $A \rightarrow \infty$.

---
    
At this point you should know why this circuit is called the **follower** circuit. 

The above calculation, although not difficult by any means, can be greatly simplified. Having an infinite gain means that, in order for the output to have a finite value, the input shall be zero.

$\qquad V_d = \frac{V_O}{A} \quad$ if $\quad A \rightarrow \infty \quad$ 
then $\quad V_d \rightarrow 0$

This is known as **virtual short circuit**. It is **short circuit** because nodes (+) and (-) are at the same voltage. It is **virtual** because it is not real, as no current can flow from one node to the other. The virtual short circuit is deduced from the ideal opamp model so it is valid when the model is valid. As we already know, the model is not always valid, like on saturated regions, so the virtual short circuit cannot always be used.
In fact, we know that the results won’t be valid if the model predicts an output voltage out of the saturation limits.

If **virtual short circuit holds**, differential voltage is zero; it turns out that the output voltage Vo shall be the same as the input Vi. It cannot get easier than that. This result should be valid if we don’t reach saturation.

## Testing the follower

Now it is time to test the follower. Mount the circuit and perform a Vo(Vi) curve using the **curveVV** command. The **curveVV** is in the DC module, so we need to import both the **slab** and the **slab_dc** modules.

As usual we also need to set the proper **path** for the calibrations.

**Execute** the following cell to connect with the board.

If, while working with this document, you have any problem with the board, you can always **reset** the board and run again this cell to reconnect.

In [ ]:
# Import the SLab module and the DC module
import slab
import slab.dc as dc

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

After we are **connected** to the board we can perform the measurement that gives us the output voltage as function of the input voltage for the follower circuit.

In [ ]:
# Voltage vs Voltage DC curve
# Sweeping DAC 1 between 0V and 3.2V in 0.1V steps
# Input is measured in ADC2
# Output is obtained in ADC 1
dc.curveVV(0,3.2,0.1,adc2=True)

We can see the **virtual shortcircuit** in action if we perform a **dcSweep** measurement. This command returns a vector with one element for the **DAC1** voltage and one additional element for **each ADC** available on the **hardware board**.

That means, that in the **F303 Zero** case, we will have a vector with **nine** elements. One for **DAC1** and eight for the **ADC** channels.

Execute the following cell to see $V_i$, $V_O$ and $V_d = V_i - V_O$

We will activate, from now on, the interactive mode so that you can zoom on the graph.  
When you end examining the graphs, close them so that you reduce the load on your PC.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

In [ ]:
# Perform the measurement
Vdac,Vo,Vi,V3,V4,*rest = slab.dcSweep(1,0,3.2)

# Show Vi and Vo as function of Vi
slab.plot1n(Vi,[Vi,Vo],'Vi and Vo','Vi (V)','Vo, Vi (V)',['Vi','Vo'])
                                                          
# Show Vd(Vi)                                                          
slab.plot11(Vi,Vi-Vo,'Vd plot','Vi (V)','Vd (V)')

In the first graph you should see that $V_i$ and $V_O$ are exactly equal. 

When you set a voltage using a **DAC**, you are expecting it to work as intended, but this is not guaranteed. So, depending on the conditions, the voltage **really set** can be different from the DAC **programmed value**. Using **ADCs**, on the contrary, always **measure** real circuit voltages.

In the second graph you see the voltage difference $V_d$ between the two amplifier inputs. The measure can have some noise, but the voltage shall be in the mV range.

Examine the results from the measurement. Does it work as expected?  
    
A follower takes an input voltage and outputs an equal voltage. The range of voltages that a follower can follow depend on two conditions:

* The range of voltages it can provide on the output

* The range of voltages it can read on the input

We know from the previous document that the **Maximum Output Voltage Swing** parameter indicates the range of voltages that can be made available on the output.

![Output Swing](images\L_OA_02\vol_voh.png)

In a similar way, the **Common Mode Input Range** parameter defines the input voltage range. It is named Common Mode because, when the virtual short circuit operates, both inputs have the same common voltage. In fact, the common voltage $V_{CM}$ is defined:

$\qquad V_{CM} = \frac{V_{(+)} + V_{(-)}}{2}$

When the opamp operates on the linear region, outside of the saturation zones, the virtual shortcircuit usually holds so:

$\qquad V_{CM} \approx V_{(+)} \approx V_{(-)}$

The Common Mode Input Range parameter can be found on the MCP6002 datasheet:

![Vcm](images\L_OA_02\vcm.png)

We can see that the common mode input range not only includes the supply rails but also goes 0.3 V outside of them.

As both the input and the output are equal and can limit the follower operation, being the **output swing range** smaller than the **common mode range**, it is the parameter that limits the follower range.

When, as happens on the MCP6002, both the Common Mode Range and the Output Swing go near both supplies we say that the opamp feature **Full-Rail Input and Output operation**. This is a very convenient feature for opamps that are powered at low voltages, as 3V, because the supply range is so small that we cannot lose any fraction of it.

In general you should not try to set the input voltages of an opamp outside of the supply range. The **Absolute Maximum Ratings** warn about that:

![AMR](images\L_OA_02\amr.png)

We can see that the opamp inputs V(+) and V(-) shall not go more than 1V outside of the supply range or the device could be permanently damaged.

The follower does not only work on DC, we can also test it in AC operation. We will test the circuit providing an input square wave with DAC 1 as shown in the circuit below. You don't need to do any change on the circuit as it alreay features all needed ADC and DAC connections. That's why it has been drawn without the typical SLab color square boxes.

![fig03](images\L_OA_02\fig03.png)

We want to test the opamp with a **square wave**. We will generate this wave with values between 1V and 2V using 100 data points.

>`slab.waveSquare(1,2,100)`

The module will respond something like:

>`100 point wave loaded
Wave frequency must be between 0.000100 and 400.00 Hz
Current frequency is 10.0 Hz
Remaining buffer space is 19900 samples`

The module informs the frequency limits for the wave on the connected Hardware Board. It also gives current wave frequency for the current sample frequency that is 1 kHz by default. The hardware board uses an unified memory for samples. The board can tipycally have a memory for 20000 samples. As 100 samples are used for the wave, we still have space for 19900 samples.

Now we set the frequency to 100 Hz.

>`slab.setWaveFrequency(100)`

That recalculates the sample information from the transient measurements. It returns current sample time Ts. As we want 100 points per cycle in a 100 Hz wave, the sample frequency fS and the sample time TS can be calculated:

$\qquad f_S = 100 \cdot 100Hz = 10kHz \qquad T_S = \frac{1}{f_S} = 100 \mu s$

When we perform the measurement the system will generate the wave sending samples, one each $100 \mu s$ to draw the waveform.

We need to instruct the SLab module the number of samples to record. In order to record 5 full waves we need 500 points. We also need to indicate the number or ADCs to read, 2 in our case.

>`slab.setTransientStorage(500,2)`

As this is a long command, you can use a shorter alias:

>`slab.tranStore(500,2)`

Now we can perform the measurement and show the results.

>`slab.wavePlot()`

All the needed code is included in the **cell** below

Also, we can plot the $V_d$ voltage to show the **virtual shortcircuit** in action.

In [ ]:
# Create a square waveform minimum of 1V, maximum of 2V and 100 sample points
slab.waveSquare(1,2,100)

# Set wave frequecy to 100 Hz
slab.setWaveFrequency(100)

# Set transient measurements to store 500 samples on two channels (ADC 1 and 2)
slab.tranStore(500,2)

# Do a transient measurement, show the result and return measurements
t,vo,vi = slab.wavePlot(returnData=True)

# Show Vd voltage as function of time
slab.plot11(t,vi-vo,'Vd voltage','Time (s)','Vd (V)')

You can probably see the ADC [quantization](https://en.wikipedia.org/wiki/Quantization_(signal_processing)) in the $V_d$ plot if you zoom near the horizontal axis.

Note how the **virtual shortcircuit** is degraded during the transitions. This is due to the fact that the amplifier has a **finite response time**.

We can also test the circuit against a sinewave if you please using the following code cell.

In [ ]:
# Response to sine wave
# We repeat some commands to make this cell independent of the previous one
slab.waveSine(1,2,100)
slab.setWaveFrequency(100)
slab.tranStore(500,2)
t,vo,vi = slab.wavePlot(returnData=True)
slab.plot11(t,vi-vo,'Vd voltage','Time (s)','Vd (V)')

Perform the transient measurements. Does the output follow the input?   

## Follower on the roller coaster

Considering all the explanations about the opamp we have made so far, connecting the potentiometer to the (-) input and the output to the (+) input should also work.

![fig04](images\L_OA_02\fig04.png)

---

**THEORETICAL TASK**  
Obtain Vo(Vi) using the Zero Order Opamp Model.    
Obtain the result for an ideal opamp that satisfies $A \rightarrow \infty$  

---

After that, mount the circuit as shown in the figure.  

Now we can measure the circuit again using an input sinewave.

In [ ]:
# Response to sine wave
# We repeat the commands to make this cell independent of the previous one
slab.waveSine(1,2,100)
slab.setWaveFrequency(100)
slab.tranStore(500,2)
slab.wavePlot()

As you can see the circuit doesn’t work as before. The reason why it doesn’t work is a question of **stability** and is related to the time response of the amplifier.
A typical example of stability is a [roller coaster](https://en.wikipedia.org/wiki/Roller_coaster). Imagine a **roller coaster** with peaks and valleys. In the following figure we see one peak at point **P2** and two valleys at points **P1** and **P3**. All these three points are **equilibrium** points because the wagon won't move if put it on any of them.

![fig05](images\L_OA_02\fig05.png)

**P1** and **P3** are two **stable** equilibrium points. If you put a train in a valley and you push it to one side, the gravity will make it return to the valley. Once a train stops in the valley, you need to provide enough energy to reach a near peak to take it out of the valley.
We say that in points P1 and P3 the system provides **negative feedback** because it tries to move the train in direction that is opposite to the disturbance force we are applying.

**P2** is an **unstable** equilibrium point. Just at the top of a peak, the train can be still, but if you give a gentle push to any side, the gravity will do the rest to move the train far from this point.
In this case we say that the system provides **positive feedback** because it enhances any action me make on the train at this point.

Our **bad follower** circuit doesn’t work because we are applying positive feedback and it is easy to see why. Suppose we have circuit $V_i$, connected to (-), at a voltage of 1V. The equilibrium condition in the linear region, as the virtual short circuit shows, will put the output node $V_O$ at 1V also. 

$$V_{i\:EQ} = 1V \qquad V_{O \: EQ} = 1V + V_{d\: EQ} \approx 1V$$

![fig06](images\L_OA_02\fig06.png)

Now, we perturbate the equilibrium by increasing $V_O$, as this voltage is applied to the positive input $V_{(+)}$, the differential voltage increases:

$$V_{(+)} = 1V + \Delta V \qquad V_d = V_{d EQ} + \Delta V$$

That yields a voltage change in the output:

$$\qquad V_O = A(V_d) = A(V_{d \: EQ} + \Delta V) 
= A \cdot V_{d \: EQ} + A \cdot \Delta V
= 1V + A \cdot \Delta V$$

So, output voltage will increase more. We had a $\Delta V$ perturbation and now we have a much higher $A \cdot \Delta V$ perturbation. That perturbation will race towards greater voltages and will only end when Vo reaches the maximum output voltage it can provide. This end point, at the $V_{O\: max}$ saturation voltage, is quite stable as $V_O$ cannot go higher. 

The circuit features **positive feedback** because any perturbation produces a response on the circuit that tries to increase it.

Observe that if the perturbation was negative, the same procedure will race down the $V_O$ voltage until we reach the $V_{O \: min}$ saturation voltage. We know the equilibrium point is not stable and we know that the operation will drift towards one of the saturation regions. But we cannot predict in which one of the two saturation regions we will end. It all depends on the circuit's **initial conditions** or the perturbation applied.

How does the first circuit, **the good one** that really follows, compares?

In this circuit $V_i$ is connected to the (+) input and the amplifier output is connected to the (-) input. 

![fig07](images\L_OA_02\fig07.png)

A gentle push on $V_O$ towards higher voltages will increase the differential $V_{(-)}$ voltage:

$$V_{(-)} = 1V + \Delta V \qquad V_d = V_{d \: EQ} + \Delta V$$

That yields a voltage decrease in the output:

$$V_O = A(V_d) = A(V_{d \: EQ} - \Delta V)
= A \cdot V_{d \: EQ} - A \cdot \Delta V 
= 1V - A \cdot \Delta V$$

The circuit features **negative feedback** because any perturbation produces a response on the circuit that tries to decrease it.

Negative feedback opamp circuits **usually** have a feedback connection from the output to the (-) input. But this is not always the case as one element on the feedback path could change the voltage polarity and require end up in (+). So, don't count on that cheap trick to always work.

Is negative feedback enough to have a stable system? Well, not exactly.

Returning to the roller coaster, imagine that we have the train still in the position P1 in the following figure.

![fig08](images\L_OA_02\fig08.png)

If we let go the train, it will go down to the valley, but when it reaches it, it has gained enough speed to climb up the other side. It won’t reach P2 because it has lost part of the energy due to friction. In the end the train will do several movements from one side of the valley to the other until it finally stops. We say that the system is **underdamped** because it oscillates around the equilibrium point before stopping.

We can be grateful of friction because it makes the train finally stop. If there was more friction, or a lower slope, the system could reach the bottom of the valley without any oscillation. In this case we say that the system is **overdamped**. The limit between this two cases is a **critically damped** system where we reach the equilibrium point as fast as possible without oscillations.
If there was no friction at all with the rails or the air, the train will eternally move between points P1 and P2 making the system a **mechanical oscillator**.

If we try too hard to compensate the perturbations with too much negative feedback, as is the roller coaster case without friction, the system will oscillate without being able to rest on any equilibrium point.

Using negative feedback, as in the first follower, makes the circuit stable, but unity gain, as obtained in the follower, is just on the limit of stability for most opamps. This is due to the fact that the follower uses a lot of negative feedback. As $V_{(-)} = V_O$, the negative feedback is as big as the output perturbation. In general, any negative feedback circuit that feeds back only a fraction of the perturbation will also be stable. However, you can’t guarantee the stability if you give a negative feedback to the input that is greater that the output perturbation.

So the general conclusions are:

* **Positive feedback** : Circuit is not stable and will drift towards one of the saturation regions.

* **Some negative feedback**: Circuit is stable

* **Too much negative feedback**: Circuit will oscillate during some time if underdamped of continuously if not damped at all.

In some systems, a little positive feedback is ok, but let's leave this simple, with only three options, for now.

## The ideal first order opamp model

As we have seen, the stability of linear opamp circuits depends on the time response of the device. We addressed the stability of the circuit relating it to positive and negative feedback in a qualitative way.
In engineering it is important to grasp the qualitative concepts of circuits, but it is also important being able to give quantitative results. And, for that, we need better models.

![fig09](images\L_OA_02\fig09.png)

The zero order basic model used in the previous project is enough to solve a linear circuit using the concept of virtual short circuit but is not able to give any result regarding its stability. This is because the zero order model assumes that the amplifier responds instantaneally to any change in the input $V_d$ voltage.

The **first order** opamp model assumes that there is some delay between the change in the input $V_d$ voltage and the change in the output $V_O$ voltage. The delay, however, is not constant, it depends on how far is the output voltage of the value where it should be.

As usual on **engineering** we solve problems using equations and this is the equation that relates the input and output of the amplifier when using a first order model:

$$\frac{dV_O(t)}{dt}=\frac{A \cdot V_d(t)-V_O(t)}{\tau}$$

In the above equation $V_d(t)$ is the amplifier input voltage and $V_O(t)$ is the amplifier output voltage.

If the output voltage is equal to $A\cdot V_d$ then the derivative is zero and the output is constant.

If the output voltage is different of the $A\cdot V_d$ value, the voltage increases or decreases to go towards the $A\cdot V_d$ value with a speed that is proportional to how far we are of the $A\cdot V_d$ value.

The constant $\tau$ just says how fast we change voltages. It's called **time constant** because its units need to be the ones of time for the equation to make sense.

In electronic circuits, this kind of equations are usually solved in the **"S"** domain using the [Laplace Transform](https://en.wikipedia.org/wiki/Laplace_transform). But this is just a tool and we really don't need it for such a simple case. In fact, not using the Laplace transform enables us to understand the circuit operation in the less abstract time domain.

In the case of the MCP6002 operational amplifier, the $A_O$ value is about 400000 and the $\tau$ value is about 63 ms. So the equation turns out to be, for this amplifier:

$$\frac{dV_O(t)}{dt}=\frac{400000 \cdot V_d(t)-V_O(t)}{63 ms}$$

The amplifier equation can be solved numerically using the [euler method](https://en.wikipedia.org/wiki/Euler_method). I know that it is not the best method, but it is really easy to understand.

This method is one of the know as **initial value** methods that solve the evolution of a system defined by differential equations when we know an initial value, like $V_O(t=0)$. Basically we solve $V_O(t)$ integrating the derivative:

$$V_O(t)=V_O(0)+\int_0^t \frac{dV_O}{dt} d\xi$$

In the **Euler** method we divide time in intervals of size $\Delta t$ and we calculate the $V_O$ value at the end of each interval from the value of the previous one, usig a crude approximation to the integral:

$$V_O(t+\Delta t)=V_O(t)+\frac{dV_O}{dt} \Delta t$$

To see that in action, let's say that currently $V_d = 0$ and $V_O = 0$ and you set at time $t=0$, $V_d(0) = 5 \mu V$, the following code shows the time evolution o the opamp output:

In [ ]:
import slab

# Start value of Vo
Vo = 0 # [V]

# Ao value
Ao = 400000

# Vd value from time t=0
Vd = 5e-6 # [V]

# Time constant
tau = 0.063 # [s]

# Time step and end time for the simulation
ts   = 1e-6 # [s]
Tend = 0.5  # [s]

# Vectors to store the results
time = []
Out  = []

# Time variable
t = 0

# Do the simulation
while t < Tend:
    # Store results
    time.append(t)
    Out.append(Vo)
    # Update Vo
    Vo = Vo + ts * (Ao*Vd - Vo)/tau
    # Update time
    t = t + ts
    
# Show the result
slab.plot11(time,Out,'First order opamp step response','Time (s)','Vo (V)')

You can see an exponential response that, if you give enough time, tends to the expected $A_O \cdot V_d$ value.

Now we can do the same in closed loop, the only difference is that the $V_d$ voltage is now:

$$V_d = V_i -  V_O$$

The following cells performs the simulation using conditions similar to the ones measured in **C#7**

In [ ]:
import numpy as np

# Start value of Vo
Vo = 1 # [V]

# Vi value before t = 0
Vi = 1 # [V]

# Time step and end time for the simulation
ts   =  1e-9   # [s]
Tend =  2e-6   # [s]

# Vectors to store the results
time2 = []
In2   = []
Out2  = []

# Time variable starts at -1 ms
t = -1e-6

# Do the simulation
while t < Tend:
    # Store results
    time2.append(t)
    In2.append(Vi)
    Out2.append(Vo)
    # Update Vo
    if t > 0: Vi = 2
    Vd = Vi - Vo
    Vo = Vo +  ts * (Ao*Vd - Vo)/tau
    # Update time
    t = t + ts
    
# Show the result
time2 = np.array(time2) * 1e6 # Use us as units
slab.plot1n(time2,[In2,Out2],'Good follower response'
            ,'Time (us)','Vo (V)',['Vi','Vo'])

We see that, when the input of the follower changes from 1V to 2V, the output also changes from 1V to 2V. Note that although the time constant of the open loop circuit was $63 ms$, the follower response is much faster, as it reaches the final value in less than $1 \mu s$.

We can try to solve the same problem using the **bad follower**. This is done in the following code cell:

In [ ]:
import numpy as np

# Start value of Vo
Vo = 1 # [V]

# Vi value before t = 0
Vi = 1 # [V]

# Time step and end time for the simulation
ts   =  1e-9   # [s]
Tend =  5e-7   # [s]

# Vectors to store the results
time3 = []
In3   = []
Out3  = []

# Time variable starts at -1 ms
t = -1e-7

# Do the simulation
while t < Tend:
    # Store results
    time3.append(t)
    In3.append(Vi)
    Out3.append(Vo)
    # Update Vo
    if t > 0: Vi = 2
    Vd = Vo - Vi  # Inputs are changed
    Vo = Vo +  ts * (Ao*Vd - Vo)/tau
    # Update time
    t = t + ts
    
# Show the result
time3 = np.array(time3) * 1e6 # Use us as units
slab.plot1n(time3,[In3,Out3],'Bad follower response'
            ,'Time (us)','Vo (V)',['Vi','Vo'])

See how the **positive feedback** makes the $V_O$ depart from the $V_i$ value. In fact, we have an exponential evolution that takes us far from solution of the **good follower**. Note that before the $V_i$ step, the circuit was in a **equilibrium** condition regardless of having a **positive feedback**. In practice, as this point is in **unstable equilibium**, any noise present in the circuit take the amplifier out of equilibrium.

Also note that, in order to see what happens, the time scale in the **C#15** bad follower graph is different from the previous good follower **C#14** case.

Off course, an operational amplifier powered between $0V$ and $3.3V$ cannot generate a negative output voltage. The following simulation is more realistic because it takes into account the saturation voltages of the opamp.

It also shows what happens when you return the $V_i$ to the initial 1V value.

In [ ]:
import numpy as np

# Start value of Vo
Vo = 1 # [V]

# Vi value before t = 0
Vi = 1 # [V]

# Time step and end time for the simulation
ts   =  1e-9   # [s]
Tend =  5e-7   # [s]

# Vectors to store the results
time4 = []
In4   = []
Out4  = []

# Time variable starts at -1 ms
t = -1e-7

# Do the simulation
while t < Tend:
    # Store results
    time4.append(t)
    In4.append(Vi)
    Out4.append(Vo)
    # Update Vo
    if t > 0: Vi = 2
    if t > 0.3e-6: Vi = 1
    Vd = Vo - Vi  # Inputs are changed
    Vo = Vo +  ts * (Ao*Vd - Vo)/tau
    # Check saturation
    if Vo > 3.3: Vo = 3.3
    if Vo < 0  : Vo = 0
    # Update time
    t = t + ts
    
# Show the result
time4 = np.array(time4) * 1e6 # Use us as units
slab.plotnn([time4,time4,[-0.1,0.0,0.5],[-0.1,0.0,0.5]]
            ,[In4,Out4,[0.0,0.0,0.0],[3.3,3.3,3.3]]
            ,'Bad follower response with saturation'
            ,'Time (us)','Vo (V)',['Vi','Vo','Sat-','Sat+'])

We can see from the simulation that once the output voltage reach the saturation it is here to stay.

Observe how, before $t=0$ we were at an **instable** equilibrium point with $V_i=1V$. Later, when at $t=0.3 \mu s$ we return to 1V, the output stays saturated.

Due to noise present in real circuits, you cannot keep the circuit in **unstable** equilibrium. The circuit will go to saturation as soon as you power it. Going to **positive** or **negative** saturation depend on the circuit power-up conditions, not in the input signal. That explains that you can get, in your measurements get both positive or negative saturation values when using **positive feedback**.

## Open loop exact solution

We have solved the open loop amplifier and the follower circuits using numerical simulation because it is easy and intuitive, but the differential equations are easy enough to solve using analytical methods.

Let's start with the **open loop** operation:

The equation for the amplifier in  **open loop** after t=0 if $V_d$ is kept constant is:

$$\frac{dV_O(t)}{dt}=\frac{A \cdot V_d-V_O(t)}{\tau}$$

We can rewrite it as:

$$\frac{dV_O}{V_O - A \cdot V_d} = -\frac{t}{\tau}$$

Now we can integrate at both sides:

$$\int \frac{dV_O}{V_O - A \cdot V_d} = -\int \frac{dt}{\tau}$$

That gives:

$$ln(V_O - A \cdot V_d) = -\int \frac{t}{\tau} + C$$

We can get rid of the logarith using exponential functions:

$$V_O - A \cdot V_d = e^{-t / \tau} \cdot e^C$$

As, $e^C$ is a constant, let's rewrite it as another **D** constant:

$$V_O = A \cdot V_d + D \cdot e^{-t / \tau}$$

In order to solve **D** we can use the initial condition for the system:

$$V_O(0) = A \cdot V_d + D \cdot e^{-t / \tau}$$

That yields:

$$D = V_O(0) - A \cdot V_d$$

So the final solution is:

$$V_O = A \cdot V_d + (V_O(0) - A \cdot V_d) \cdot e^{-t / \tau}$$

We can draw this analitical solution together with the numerical solution previously obtained:

In [ ]:
# Analytical solution
Vd = 5e-6 # [V]
time = np.array(time)
Out_A = Ao*Vd - Ao*Vd*np.exp(-time/tau)

# Compare to simulation
slab.plot1n(time,[Out,Out_A],'First order opamp step response'
            ,'Time (s)','Vo (V)',['Simulation','Analytical'])

You can see that the exact analytical solution gives the same values as the previous simulation.

## Good follower exact solution

The **good follower** can also be solved analytically. We also have:

$$\frac{dV_O(t)}{dt}=\frac{A \cdot V_d-V_O(t)}{\tau}$$

But now $V_d$ is not constant but dependent on $V_i$ and $V_O(t)$:

$$V_d(t) = V_i - V_O(t)$$

Having V_i constant for t > 0, we obtain:

$$\frac{dV_O(t)}{dt}=\frac{A \cdot V_i - (1+A) V_O(t)}{\tau}$$

From that:

$$-\frac{dt}{\tau / (1+A)} = \frac {dV_O}{V_O -\frac{A}{1+A}V_i}
\approx \frac {dV_O}{V_O - V_i}$$

Solving the equation we get:

$$V_O = V_i + (V_O(0)-V_i)e^{-t \cdot (1+A)/ \tau}$$

As we have seen on the simulations, the time constant is much smaller on closed loop than in open loop.

We can draw this analitical solution together with the numerical solution previously obtained:

In [ ]:
# Analytical solution
time2 = np.array(time2)
tau2 = tau/(1+Ao)
# Solution valid for t > 0 (time is in us)
Out2_A = 2.0 - 1.0 * np.exp(-time2*1e-6/tau2)
# Add solution valid for t <= 0
Out2_A = np.array([1 if t < 0 else x for (t,x) in zip(time2,Out2_A)])

# Compare to simulation
slab.plot1n(time2,[In2,Out2,Out2_A],'Good follower step response'
            ,'Time (s)','Vo (V)',['Step','Simulation','Analytical'])

Again the simulation agrees with analytical solution.

## Bad follower exact solution

Finally we can also solve the **bad follower** equations. They are the same of the good foollower and only changes the $V_d$ sign:

$$\frac{dV_O(t)}{dt}=\frac{A \cdot  V_d - V_O(t)}{\tau}
\qquad V_d(t) = V_O(t) - V_i$$

So, V_i constant for t > 0, we obtain:

$$\frac{dV_O(t)}{dt}=\frac{(A-1)V_O - A \cdot Vi}{\tau}$$

From that:

$$\frac{dt}{\tau / (A-1)} = \frac {dV_O}{V_O -\frac{A}{A-1}V_i}
\approx \frac {dV_O}{V_O - V_i}$$

Solving the equation we get:

$$V_O = V_i + (V_O(0)-V_i)e^{t \cdot (A-1)/ \tau}$$

Observe that the exponential has a positive exponent, so it will tend to go towards infinity.

We can draw this analitical solution together with the numerical solution previously obtained:

In [ ]:
# Analytical solution
time3 = np.array(time3)
tau3 = tau/(Ao-1)
# Solution valid for t > 0 (time is in us)
Out3_A = 2.0 - 1.0 * np.exp(time3*1e-6/tau3)
# Add solution valid for t <= 0
Out3_A = np.array([1 if t < 0 else x for (t,x) in zip(time3,Out3_A)])

# Compare to simulation
slab.plot1n(time3,[In3,Out3,Out3_A],'Bad follower step response'
            ,'Time (s)','Vo (V)',['Step','Simulation','Analytical'])

You can see that, also in this case, there is a very good agreement between the simulation and the analytical solution.

## Last comments

In this document we have seen how the **zero order model**, and the associated **virtual shortcircuit** condition, can help us to obtain the response of an operational amplifier.

We have seen that, due to the time response of the amplifier, the result is only valid when there is an **stable** equilibrium point thanks to the **negative feedback**.

Working in the time domain we have demonstrated, both using simulations and analytical calculations, that the **negative feedback** gives a proper follower circuit whereas using **positive feedback** gives us a bad follower circuit that saturates.

Calculating in the time domain is good because it gives good intuitive explanations of the circuit operation. The problem is that the time domain requires us to work on differential equations that are not always easy to solve. That's why we commonly use the **Laplace transform** as we will see in the following document.

## References

**SLab Python References**  
Those are the reference documents for the SLab Python modules. They describe the commands that can be carried out after importing each module.   
They should be available in the **SLab/Doc** folder.

**TinyCad**  
Circuit images on this document have been drawn using the free software TinyCad  
https://sourceforge.net/projects/tinycad/

**Matplotlib**
All the functions plots have been generated using the Matplotlib package.  
https://matplotlib.org/


## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">